In [ ]:
### Parsing CSV Files

In [26]:
# Your task is to read the input DATAFILE line by line, and for the first 10 lines (not including the header)
# split each line on "," and then for each line, create a dictionary
# where the key is the header title of the field, and the value is the value of that field in the row.
# The function parse_file should return a list of dictionaries,
# each data line in the file being a single list entry.
# Field names and values should not contain extra whitespace, like spaces or newline characters.
# You can use the Python string method strip() to remove the extra whitespace.
# You have to parse only the first 10 data lines in this exercise,
# so the returned list should have 10 entries!
import os

DATADIR = ""
DATAFILE = "beatles-discography.csv"


def parse_file(datafile):
    data = []
    counter = 0
    max_entries = 10
    
    with open(datafile, "r") as f:
        for line in f:
            if counter == 0:
                header = [x.strip() for x in line.split(',')]
            elif counter <= max_entries:
                data.append(dict(zip(header, [x.strip() for x in line.split(',')])))
            
            counter += 1
    
    return data


def test():
    # a simple test of your implemetation
    datafile = os.path.join(DATADIR, DATAFILE)
    d = parse_file(datafile)
    firstline = {'Title': 'Please Please Me', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '22 March 1963', 'US Chart Position': '-', 'RIAA Certification': 'Platinum', 'BPI Certification': 'Gold'}
    tenthline = {'Title': '', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '10 July 1964', 'US Chart Position': '-', 'RIAA Certification': '', 'BPI Certification': 'Gold'}

    assert d[0] == firstline
    assert d[9] == tenthline

    
test()

### Reading Excel files

In [25]:
#!/usr/bin/env python
"""
Your task is as follows:
- read the provided Excel file
- find and return the min, max and average values for the COAST region
- find and return the time value for the min and max entries
- the time values should be returned as Python tuples

Please see the test function for the expected return format
"""
import numpy as np
import xlrd
from zipfile import ZipFile
datafile = "2013_ERCOT_Hourly_Load_Data.xls"


def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    ### example on how you can get the data
    #sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]

    ### other useful methods:
    # print "\nROWS, COLUMNS, and CELLS:"
    # print "Number of rows in the sheet:", 
    # print sheet.nrows
    # print "Type of data in cell (row 3, col 2):", 
    # print sheet.cell_type(3, 2)
    # print "Value in cell (row 3, col 2):", 
    # print sheet.cell_value(3, 2)
    # print "Get a slice of values in column 3, from rows 1-3:"
    # print sheet.col_values(3, start_rowx=1, end_rowx=4)

    # print "\nDATES:"
    # print "Type of data in cell (row 1, col 0):", 
    # print sheet.cell_type(1, 0)
    # exceltime = sheet.cell_value(1, 0)
    # print "Time in Excel format:",
    # print exceltime
    # print "Convert time to a Python datetime tuple, from the Excel float:",
    # print xlrd.xldate_as_tuple(exceltime, 0)
    
    coast_values = np.array(sheet.col_values(1, start_rowx=1))
    
    coast_max_value_row = coast_values.argmax()
    maxtime_as_float = sheet.cell_value(coast_max_value_row + 1, 0)
    maxtime = xlrd.xldate_as_tuple(maxtime_as_float, 0)
    
    coast_min_value_row = coast_values.argmin()
    mintime_as_float = sheet.cell_value(coast_min_value_row + 1, 0)
    mintime = xlrd.xldate_as_tuple(mintime_as_float, 0)
    
    data = {
            'maxtime': maxtime,
            'maxvalue': coast_values.max(),
            'mintime': mintime,
            'minvalue': coast_values.min(),
            'avgcoast': coast_values.mean()
    }
    return data


def test():
    open_zip(datafile)
    data = parse_file(datafile)
    print data
    
    assert data['maxtime'] == (2013, 8, 13, 17, 0, 0)
    assert round(data['maxvalue'], 10) == round(18779.02551, 10)


test()

{'mintime': (2013, 2, 3, 4, 0, 0), 'maxtime': (2013, 8, 13, 17, 0, 0), 'avgcoast': 10976.933460679784, 'maxvalue': 18779.025510000003, 'minvalue': 6602.1138989999818}


### JSON playground

In [3]:
"""
To experiment with this code freely you will have to run this code locally.
Take a look at the main() function for an example of how to use the code. We
have provided example json output in the other code editor tabs for you to look
at, but you will not be able to run any queries through our UI.
"""
import json
import requests

BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"


# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    """
    This is the main function for making queries to the musicbrainz API. The
    query should return a json document.
    """
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    """
    This adds an artist name to the query parameters before making an API call
    to the function above.
    """
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    """
    After we get our output, we can use this function to format it to be more
    readable.
    """
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data


def main():
    """
    Below is an example investigation to help you get started in your
    exploration. Modify the function calls and indexing below to answer the
    questions on the next quiz.

    HINT: Note how the output we get from the site is a multi-level JSON
    document, so try making print statements to step through the structure one
    level at a time or copy the output to a separate output file. Experimenting
    and iteration will be key to understand the structure of the data!
    """

    # Query for information in the database about bands named Nirvana
    results = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
    pretty_print(results)

    # Isolate information from the 4th band returned (index 3)
    print "\nARTIST:"
    pretty_print(results["artists"][3])

    # Query for releases from that band using the artist_id
    artist_id = results["artists"][3]["id"]
    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]

    # Print information about releases from the selected band
    print "\nONE RELEASE:"
    pretty_print(releases[0], indent=2)

    release_titles = [r["title"] for r in releases]
    print "\nALL TITLES:"
    for t in release_titles:
        print t

#if __name__ == '__main__':
#    main()
main()

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json
{
    "artists": [
        {
            "area": {
                "id": "6a264f94-6ff1-30b1-9a81-41f7bfabd616", 
                "name": "Finland", 
                "sort-name": "Finland"
            }, 
            "country": "FI", 
            "disambiguation": "Early 1980's Finnish punk band", 
            "id": "85af0709-95db-4fbc-801a-120e9f4766d0", 
            "life-span": {
                "ended": null
            }, 
            "name": "Nirvana", 
            "score": "100", 
            "sort-name": "Nirvana", 
            "tags": [
                {
                    "count": 1, 
                    "name": "punk"
                }, 
                {
                    "count": 1, 
                    "name": "finland"
                }
            ], 
            "type": "Group"
        }, 
        {
            "disambiguation": "French band from Martigues, activ during the 70s.", 
      

requesting http://musicbrainz.org/ws/2/artist/9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6?fmt=json&inc=releases

ONE RELEASE:
{
  "barcode": null, 
  "country": "GB", 
  "date": "1969", 
  "disambiguation": "", 
  "id": "0b44cb36-550a-491d-bfd9-8751271f9de7", 
  "packaging": null, 
  "packaging-id": null, 
  "quality": "normal", 
  "release-events": [
    {
      "area": {
        "disambiguation": "", 
        "id": "8a754a16-0027-3a29-b6d7-2b40ea0481ed", 
        "iso-3166-1-codes": [
          "GB"
        ], 
        "name": "United Kingdom", 
        "sort-name": "United Kingdom"
      }, 
      "date": "1969"
    }
  ], 
  "status": "Official", 
  "status-id": "4e304316-386d-3409-af2e-78857eec5cfe", 
  "text-representation": {
    "language": "eng", 
    "script": "Latn"
  }, 
  "title": "To Markos III"
}

ALL TITLES:
To Markos III
Travelling on a Cloud
Songs Of Love And Praise
Songs of Love and Praise
Songs of Love and Praise
Secret Theatre
The Story of Simon Simopath
Me And My Friend


### Exploring JSON

- How many bands named "First Aid Kit""?
- Begin_area name for Queen?
- Spanish alias for Beatles?
- Nirvana disambiguation?
- When was One Direction formed?

In [4]:
first_aid_bands = query_by_name(ARTIST_URL, query_type["simple"], "First Aid Kit")
count = 0
#print first_aid_bands['artists']
for artist in first_aid_bands['artists']:
    if artist['name'] == "First Aid Kit":
        count += 1
print "How many bands named First Aid Kit?", count #first_aid_bands["count"]

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AFirst+Aid+Kit&fmt=json
How many bands named First Aid Kit? 2


In [24]:
queens_bands = query_by_name(ARTIST_URL, query_type["simple"], "Queen")
print "Begin_area name for Queen?", queens_bands["artists"][2]["begin-area"]["name"]

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AQueen&fmt=json
Begin_area name for Queen? London


In [36]:
beatles_bands = query_by_name(ARTIST_URL, query_type["simple"], "Beatles")
beatles_aliases = beatles_bands["artists"][7]["aliases"]
for alias in beatles_aliases:
    if alias["locale"] == "es":
        print "Spanish alias for Beatles?", alias["name"]

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ABeatles&fmt=json
Spanish alias for Beatles? Los Beatles


In [50]:
nirvanas = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
print "Nirvana disambiguation?", nirvanas["artists"][4]["disambiguation"]

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json
Nirvana disambiguation? 90s US grunge band


In [57]:
one_direction_bands = query_by_name(ARTIST_URL, query_type["simple"], "One Direction")

print "When was One Direction formed?", one_direction_bands["artists"][0]["life-span"]["begin"]

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AOne+Direction&fmt=json
When was One Direction formed? 2010-07
